In [94]:
import pandas as pd 
import numpy as np 
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime

In [37]:
def save_json(data, filename):
    """
    Write a Python object to a JSON file.
    
    Args:
        data: The Python object to be serialized and written to the file.
        filename (str): The name of the JSON file where the data will be saved.
        
    Returns:
        None
    """
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [38]:

def read_csv_to_dataframe(file_path):
    try:
        # Read the CSV file into a Pandas DataFrame
        dataframe = pd.read_csv(file_path)
        dataframe.fillna(method='bfill', inplace=True)
        return dataframe
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [50]:
import math 

def fill_NANS(df): 

    output_df = df.copy() 

    columns = ['o2sat']

    medians = {} 

    for col in columns: 
        col_median = df[col].median()
        medians[col] = col_median 

    for index, row in df.iterrows(): 
        for col in columns: 
            if math.isnan(row[col]): 
                df.at[index, col] = medians[col]

    return df

In [51]:
patients_df = read_csv_to_dataframe("data/patients.csv")
# prescriptions_df = read_csv_to_dataframe("data/prescriptions.csv")
inputevents_df = read_csv_to_dataframe("data/inputevents.csv")
# procedureevents_df = read_csv_to_dataframe("data/procedureevents.csv")
# d_icd_diagnoses_df = read_csv_to_dataframe("data/d_icd_diagnoses.csv")
# triage_df = read_csv_to_dataframe("data/triage.csv")
vitalsign_df = read_csv_to_dataframe("data/vitalsign.csv")

/var/folders/rl/fmlvy3bj3b15n9nltj9m084c0000gn/T/ipykernel_1693/440882446.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe.fillna(method='bfill', inplace=True)
/var/folders/rl/fmlvy3bj3b15n9nltj9m084c0000gn/T/ipykernel_1693/440882446.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe.fillna(method='bfill', inplace=True)
/var/folders/rl/fmlvy3bj3b15n9nltj9m084c0000gn/T/ipykernel_1693/440882446.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe.fillna(method='bfill', inplace=True)


In [43]:
inputevents_df

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,66056,2180-07-23 21:10:00,2180-07-23 21:11:00,2180-07-23 21:10:00,226452,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.00000
1,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:01:00,2180-07-23 18:56:00,226452,200.000000,ml,...,Main order parameter,Bolus,39.4,200.0,ml,0,0,FinishedRunning,200.0,200.00000
2,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,220862,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.00000
3,10000032,29079034,39553978,88981,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,220862,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.00000
4,10000032,29079034,39553978,88981,2180-07-23 18:56:00,2180-07-23 18:57:00,2180-07-23 18:56:00,226452,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978888,19999987,23865745,36195440,90295,2145-11-02 23:28:00,2145-11-03 09:32:00,2145-11-02 23:28:00,225942,0.755000,mg,...,Main order parameter,Continuous Med,60.0,50.0,ml,0,0,Paused,2.5,75.00000
8978889,19999987,23865745,36195440,90295,2145-11-02 23:28:00,2145-11-03 09:32:00,2145-11-02 23:28:00,225943,15.100000,ml,...,Mixed solution,Continuous Med,60.0,50.0,ml,0,0,Paused,50.0,1.50000
8978890,19999987,23865745,36195440,90295,2145-11-02 23:38:00,2145-11-03 12:55:00,2145-11-03 14:29:00,225158,1000.000037,ml,...,Main order parameter,Continuous IV,60.0,1000.0,ml,0,0,FinishedRunning,1000.0,75.28231
8978891,19999987,23865745,36195440,91879,2145-11-04 20:35:00,2145-11-04 20:36:00,2145-11-04 20:35:00,226452,200.000000,ml,...,Main order parameter,Bolus,94.0,200.0,ml,0,0,FinishedRunning,200.0,200.00000


In [52]:
data_pv = pd.merge(patients_df, vitalsign_df, on='subject_id', how='inner')

In [53]:
fill_NANS(data_pv)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:36:00,98.4,83.0,24.0,97.0,90.0,51.0,Sinus Bradycardia,0
1,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:43:00,98.4,85.0,22.0,98.0,76.0,39.0,Sinus Bradycardia,0
2,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:45:00,98.4,84.0,22.0,97.0,75.0,39.0,Sinus Bradycardia,0
3,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 17:56:00,98.4,84.0,20.0,99.0,86.0,51.0,Sinus Bradycardia,0
4,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0,Sinus Bradycardia,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564605,19999828,F,46,2147,2017 - 2019,2164-09-17,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0,Sinus Tachycardia,0
1564606,19999914,F,49,2158,2017 - 2019,NaN,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0,Sinus Tachycardia,0
1564607,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 19:40:00,99.3,112.0,18.0,98.0,118.0,83.0,Sinus Tachycardia,unable
1564608,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 20:11:00,99.3,111.0,18.0,98.0,123.0,82.0,Sinus Tachycardia,unable


In [54]:
data_pv

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:36:00,98.4,83.0,24.0,97.0,90.0,51.0,Sinus Bradycardia,0
1,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:43:00,98.4,85.0,22.0,98.0,76.0,39.0,Sinus Bradycardia,0
2,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:45:00,98.4,84.0,22.0,97.0,75.0,39.0,Sinus Bradycardia,0
3,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 17:56:00,98.4,84.0,20.0,99.0,86.0,51.0,Sinus Bradycardia,0
4,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0,Sinus Bradycardia,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564605,19999828,F,46,2147,2017 - 2019,2164-09-17,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0,Sinus Tachycardia,0
1564606,19999914,F,49,2158,2017 - 2019,NaN,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0,Sinus Tachycardia,0
1564607,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 19:40:00,99.3,112.0,18.0,98.0,118.0,83.0,Sinus Tachycardia,unable
1564608,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 20:11:00,99.3,111.0,18.0,98.0,123.0,82.0,Sinus Tachycardia,unable


In [8]:
patients_df

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,2161-02-13
2,10000068,F,19,2160,2008 - 2010,2161-02-13
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,2187-11-12
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,2164-09-17
299708,19999829,F,28,2186,2008 - 2010,2164-09-17
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,NaN


In [55]:
unique_rhythms = data_pv['rhythm'].unique()
n_rhythms = len(unique_rhythms)
rhythms_mapping = {k:v for k,v in zip(unique_rhythms, range(n_rhythms))}
def map_rhythm(r): 
    return rhythms_mapping[r]

In [56]:
features = ["gender", "anchor_age", "temperature", "heartrate", "resprate", "o2sat", "sbp", "dbp", "rhythm"]

def feature_map(row): 
    """
    row: row from dataframe 'data_pv' 

    return a feature mapping of the row (i.e. row of matrix M)
    """
    r = [] 
    for feature in features: 
        if feature == "gender":
            r.append(0 if row[feature] == "M" else 1)
        elif feature == "rhythm": 
            r.append(map_rhythm(row[feature]))
        else:
            r.append(row[feature])

    return r 

def construct_M(df):
    M = []
    for _, row in df.iterrows():
        r = feature_map(row)
        M.append(r)
    
    save_json(M, "data/process/M.json")
    return np.array(M)

In [57]:
M = construct_M(data_pv)

In [59]:
M

array([[  1. ,  52. ,  98.4, ...,  90. ,  51. ,   0. ],
       [  1. ,  52. ,  98.4, ...,  76. ,  39. ,   0. ],
       [  1. ,  52. ,  98.4, ...,  75. ,  39. ,   0. ],
       ...,
       [  1. ,  57. ,  99.3, ..., 118. ,  83. ,   4. ],
       [  1. ,  57. ,  99.3, ..., 123. ,  82. ,   4. ],
       [  1. ,  57. ,  99.3, ..., 113. ,  79. ,   4. ]])

In [60]:
def discretize_S(M, n_clusters=100, random_state=42): 
    """
    M: matrix representation of data 

    returns a model that takes in a feature set D -> state s from 0 to K - 1
    where K is the number of clusters 
    """

    # instantiate model and fit data
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(M)

    # return model 
    return kmeans

In [61]:
state_model = discretize_S(M)

/Users/jaydenpersonnat/MI-IRL/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [62]:
inputevents_df

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,66056,2180-07-23 21:10:00,2180-07-23 21:11:00,2180-07-23 21:10:00,226452,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.00000
1,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:01:00,2180-07-23 18:56:00,226452,200.000000,ml,...,Main order parameter,Bolus,39.4,200.0,ml,0,0,FinishedRunning,200.0,200.00000
2,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,220862,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.00000
3,10000032,29079034,39553978,88981,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,220862,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.00000
4,10000032,29079034,39553978,88981,2180-07-23 18:56:00,2180-07-23 18:57:00,2180-07-23 18:56:00,226452,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978888,19999987,23865745,36195440,90295,2145-11-02 23:28:00,2145-11-03 09:32:00,2145-11-02 23:28:00,225942,0.755000,mg,...,Main order parameter,Continuous Med,60.0,50.0,ml,0,0,Paused,2.5,75.00000
8978889,19999987,23865745,36195440,90295,2145-11-02 23:28:00,2145-11-03 09:32:00,2145-11-02 23:28:00,225943,15.100000,ml,...,Mixed solution,Continuous Med,60.0,50.0,ml,0,0,Paused,50.0,1.50000
8978890,19999987,23865745,36195440,90295,2145-11-02 23:38:00,2145-11-03 12:55:00,2145-11-03 14:29:00,225158,1000.000037,ml,...,Main order parameter,Continuous IV,60.0,1000.0,ml,0,0,FinishedRunning,1000.0,75.28231
8978891,19999987,23865745,36195440,91879,2145-11-04 20:35:00,2145-11-04 20:36:00,2145-11-04 20:35:00,226452,200.000000,ml,...,Main order parameter,Bolus,94.0,200.0,ml,0,0,FinishedRunning,200.0,200.00000


In [63]:
inputevents_sample = inputevents_df.sample(n=500000, random_state = 42)

In [67]:
n_actions = len(inputevents_df['ordercategorydescription'].unique())
actions = inputevents_df['ordercategorydescription'].unique() 

action_mapping = {k:v for k, v in zip(actions, range(n_actions))}

def action_map(I):
    """
    I: event from inputevent dataframe 

    returns an action that represents event 
    """

    return action_mapping[I]

In [72]:
data_pv

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:36:00,98.4,83.0,24.0,97.0,90.0,51.0,Sinus Bradycardia,0
1,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:43:00,98.4,85.0,22.0,98.0,76.0,39.0,Sinus Bradycardia,0
2,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:45:00,98.4,84.0,22.0,97.0,75.0,39.0,Sinus Bradycardia,0
3,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 17:56:00,98.4,84.0,20.0,99.0,86.0,51.0,Sinus Bradycardia,0
4,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0,Sinus Bradycardia,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564605,19999828,F,46,2147,2017 - 2019,2164-09-17,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0,Sinus Tachycardia,0
1564606,19999914,F,49,2158,2017 - 2019,NaN,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0,Sinus Tachycardia,0
1564607,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 19:40:00,99.3,112.0,18.0,98.0,118.0,83.0,Sinus Tachycardia,unable
1564608,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 20:11:00,99.3,111.0,18.0,98.0,123.0,82.0,Sinus Tachycardia,unable


In [71]:
inputevents_sample

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
2670315,12980052,27167994,37484422,54251,2130-05-01 13:40:00,2130-05-01 13:55:00,2130-05-01 14:00:00,225943,62.499998,ml,...,Mixed solution,Continuous Med,63.6,62.5,ml,0,0,FinishedRunning,62.500000,250.000000
1759629,11951833,26441306,32430524,52967,2201-10-08 14:56:00,2201-10-08 15:03:00,2201-10-08 14:56:00,221906,0.144156,mg,...,Main order parameter,Continuous Med,86.4,250.0,ml,0,0,ChangeDose/Rate,7.084240,0.240000
3085865,13461807,27122933,37796943,98277,2142-10-05 17:19:00,2142-10-05 18:15:00,2142-10-05 17:01:00,220949,46.834703,ml,...,Mixed solution,Continuous Med,62.7,500.0,ml,0,0,Stopped,500.000000,49.999996
2571162,12860941,25544823,38151958,1215,2135-12-03 03:44:00,2135-12-03 03:45:00,2135-12-03 03:44:00,225158,100.000000,ml,...,Main order parameter,Bolus,100.0,100.0,ml,0,0,FinishedRunning,100.000000,100.000000
4507630,15031111,27917559,37624914,59086,2152-12-30 13:00:00,2152-12-30 13:30:00,2152-12-30 13:15:00,225158,49.999999,ml,...,Main order parameter,Continuous IV,84.0,50.0,ml,0,0,FinishedRunning,50.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400386,18249179,21953437,32363848,24777,2188-02-05 23:53:00,2188-02-05 23:54:00,2188-02-05 23:53:00,220949,100.000000,ml,...,Mixed solution,Drug Push,70.0,100.0,ml,0,0,FinishedRunning,100.000000,0.000000
8484112,19478022,21072589,38738612,68986,2131-09-25 14:57:00,2131-09-25 14:58:00,2131-09-25 16:03:00,225154,1.000000,mg,...,Main order parameter,Drug Push,55.0,25.0,ml,0,0,FinishedRunning,1.000000,0.000000
3639242,14048242,20218868,37322924,94669,2128-08-19 16:48:00,2128-08-19 16:49:00,2128-08-19 16:48:00,221833,1.000000,mg,...,Main order parameter,Drug Push,78.0,500.0,ml,0,0,FinishedRunning,1.000000,1.000000
7641858,18520455,25408103,32146543,8388,2172-05-05 12:23:00,2172-05-05 12:24:00,2172-05-05 12:23:00,226452,120.000000,ml,...,Main order parameter,Bolus,96.5,120.0,ml,0,0,FinishedRunning,120.000000,120.000000


In [104]:

def find_patient_events(events_df): 
    subject_events = {} 
    
    for _, event in events_df.iterrows(): 
        subject = event['subject_id']

        patient_event = { 'caregiver': event['caregiver_id'], 'starttime': event['starttime'], 'endtime': event['endtime'], 'action': action_map(event['ordercategorydescription']) }

        if subject not in subject_events: 
            value = [patient_event]
            subject_events[subject] = value 
        else: 
            subject_events[subject].append(patient_event)

    # sort these by time
    for s in subject_events: 
        subject_events[s] = sorted(subject_events[s], key=lambda x: x['starttime'])
    return subject_events

def group_timestamps_by_day(events):
    timestamp_dict = {}

    for event in events:
        # Parse the timestamp string into a datetime object
        dt = datetime.strptime(event['starttime'], '%Y-%m-%d %H:%M:%S')
        
        # Extract the day part (date) from the datetime object
        day = dt.date()
        
        # Convert the date back to a string
        day_str = day.strftime('%Y-%m-%d')
        
        # Add the timestamp to the corresponding day's list in the dictionary
        if day_str not in timestamp_dict:
            timestamp_dict[day_str] = [event]
        else:
            timestamp_dict[day_str].append(event)

    return timestamp_dict

def find_patient_vitals(data_pv): 
    subject_vitals = {} 
    
    for _, vitals in data_pv.iterrows(): 
        subject = vitals['subject_id']

        state = state_model.predict(np.array([feature_map(vitals)]))
        patient_vital = { 'charttime': vitals['charttime'], 'state': state }

        if subject not in subject_vitals: 
            value = [patient_vital]
            patient_vital[subject] = value 
        else: 
            subject_vitals[subject].append(patient_vital)

    # sort these by time
    for s in subject_vitals: 
        subject_vitals[s] = sorted(subject_vitals[s], key=lambda x: x['charttime'])
    return subject_vitals


def trajs_from_patient(t_series): 
    """  
    t_series: time series of events for particular patient of the form [{ keys: value }]
    """

    day_timestamps = group_timestamps_by_day(t_series)

    return day_timestamps

In [ ]:
def construct_trajectories(events_df, data_pv): 
    """
    events_df: dataframe containing actions performed on patients 
    data_pv: dataframe containing vital readings 
    """
    
    subject_events = {} # key - subject_id, value: [event1, event2, ...]
                    # each event of form { 'caregiver', 'startime', 'endtime', action }
    

    for index, event in events_df.iterrows(): 
        subject = event['subject_id']

        patient_event = { 'caregiver': event['caregiver_id'], 'starttime': event['starttime'], 'endtime': event['endtime'], 'action': action_map(event['ordercategorydescription']) }

        if subject not in subject_events: 
            value = [patient_event]
            subject_events[subject] = value 
        else: 
            subject_events[subject].append(patient_event)

    return subject_events

In [84]:
patient_events = find_patient_events(inputevents_sample)

In [105]:
patient_vitals = find_patient_vitals(data_pv)

In [96]:
group_timestamps_by_day(patient_events[12980052])

{'2130-04-27': [{'caregiver': 25954,
   'starttime': '2130-04-27 07:23:00',
   'endtime': '2130-04-27 07:24:00',
   'action': 2}],
 '2130-04-28': [{'caregiver': 23711,
   'starttime': '2130-04-28 00:45:00',
   'endtime': '2130-04-28 10:43:00',
   'action': 1},
  {'caregiver': 23711,
   'starttime': '2130-04-28 02:15:00',
   'endtime': '2130-04-28 02:28:00',
   'action': 3},
  {'caregiver': 23711,
   'starttime': '2130-04-28 03:47:00',
   'endtime': '2130-04-28 07:48:00',
   'action': 3},
  {'caregiver': 23711,
   'starttime': '2130-04-28 04:42:00',
   'endtime': '2130-04-28 05:01:00',
   'action': 3},
  {'caregiver': 82508,
   'starttime': '2130-04-28 10:43:00',
   'endtime': '2130-04-28 20:37:00',
   'action': 1},
  {'caregiver': 82508,
   'starttime': '2130-04-28 11:18:00',
   'endtime': '2130-04-28 11:27:00',
   'action': 3},
  {'caregiver': 82508,
   'starttime': '2130-04-28 17:12:00',
   'endtime': '2130-04-28 19:54:00',
   'action': 3},
  {'caregiver': 23711,
   'starttime': '213